In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/MuhammadYusufAndrika/tripskuy-capstone/refs/heads/main/fix_tourism.csv")
df.head()

,id_place,name_place,price_place,rating_place,id_city,id_category,name_category,name_city,desc_place
0,6,Taman Impian Jaya Ancol,25000,4.5,212,2,Taman Hiburan,Jakarta Utara,Taman Impian Jaya Ancol merupakan sebuah objek...
1,47,Taman Situ Lembang,0,4.5,216,2,Taman Hiburan,Jakarta Pusat,Taman Situ Lembang adalah sebuah taman kota ya...
2,66,Museum Layang-layang,10000,4.5,215,1,Budaya,Jakarta Selatan,Museum Layang-Layang adalah sebuah museum yang...
3,1,Monumen Nasional,20000,4.6,216,1,Budaya,Jakarta Pusat,Monumen Nasional atau yang populer disingkat d...
4,2,Kota Tua,0,4.6,213,1,Budaya,Jakarta Barat,"Kota tua di Jakarta, yang juga bernama Kota Tu..."


In [3]:
#encode/mengubah category dan city untuk inputan agar mudah di proses
le_category = LabelEncoder()
le_city = LabelEncoder()
df['category_encoded'] = le_category.fit_transform(df['name_category'])
df['city_encoded'] = le_city.fit_transform(df['name_city'])

In [4]:
#fitur prediksi / inputan
X = df[['category_encoded', 'price_place', 'city_encoded']]
#target prediksi
y = df['id_place']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
#scaler sebagai alat yang akan melakukan standardisasi.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
#convert data ke tensorflow
X_train_tf = tf.convert_to_tensor(X_train_scaled, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train.values, dtype=tf.float32)
X_test_tf = tf.convert_to_tensor(X_test_scaled, dtype=tf.float32)
y_test_tf = tf.convert_to_tensor(y_test.values, dtype=tf.float32)

In [7]:
#create model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(
    #menggunakan optimizer adam
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

In [9]:
# mencegah overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [10]:
model.fit(
    X_train_tf,
    y_train_tf,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=0
)

Available Categories:
['Taman Hiburan' 'Budaya' 'Cagar Alam' 'Bahari' 'Pusat Perbelanjaan'
 'Tempat Ibadah']


Available Cities:
['Jakarta Utara' 'Jakarta Pusat' 'Jakarta Selatan' 'Jakarta Barat'
 'Jakarta Timur' 'Kepulauan Seribu' 'Yogyakarta' 'Bandung' 'Semarang'
 'Surabaya']

In [ ]:
def main():
    while True:
        print("\n=== Tourism Destination Recommendations ===")

        print("\nAvailable Categories:")
        print(df['name_category'].unique())
        category = input("Enter Tourism Category: ")

        print("\nAvailable Cities:")
        print(df['name_city'].unique())
        city = input("Enter Destination City: ")

        while True:
            try:
                price = float(input("Enter Price (Rp): "))
                break
            except ValueError:
                print("Please enter a valid number for price!")

        category_encoded = le_category.transform([category])[0]
        city_encoded = le_city.transform([city])[0]
        input_data = scaler.transform([[category_encoded, price, city_encoded]])
        input_tensor = tf.convert_to_tensor(input_data, dtype=tf.float32)
        predicted_rating = float(model.predict(input_tensor, verbose=0)[0][0])
        print(f"\nPredicted Rating: {predicted_rating:.2f}")

        filtered_df = df[
            (df['name_category'] == category) &
            (df['name_city'] == city)
        ]

        if filtered_df.empty:
            print("No destinations match the criteria.")
        else:
            filtered_df['price_diff'] = abs(filtered_df['price_place'] - price)
            recommendations = filtered_df.nsmallest(5, 'price_diff')[['id_place', 'name_place', 'price_place', 'rating_place' , 'desc_place']].reset_index(drop=True)
            print("\nRecommended Destinations:")
            print(recommendations)

        if input("\nWant to search for more recommendations? (y/n): ").lower() != 'y':
            break


if __name__ == "__main__":
    main()


=== Tourism Destination Recommendations ===

Available Categories:
['Taman Hiburan' 'Budaya' 'Cagar Alam' 'Bahari' 'Pusat Perbelanjaan'
 'Tempat Ibadah']


In [ ]:
import os

# Save the model
model.save('tourism_recommendation_model.keras') # Add .keras extension to the filename

# Check if the model directory exists, if not create one
model_directory = 'tourism_recommendation_model'
if not os.path.exists(model_directory):
    os.makedirs(model_directory)

# Save the label encoders and scaler
import joblib
joblib.dump(le_category, os.path.join(model_directory, 'le_category.pkl'))
joblib.dump(le_city, os.path.join(model_directory, 'le_city.pkl'))
joblib.dump(scaler, os.path.join(model_directory, 'scaler.pkl'))